# dataflow 
> extract data, assemble into dataframe, analyse, training and inferring 

In [ ]:
#| default_exp dataflow

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv
from dotenv import dotenv_values
import pandas_gbq

In [ ]:
# os.environ

In [ ]:

load_dotenv("../.env")

In [ ]:
os.environ

In [ ]:
# os.getenv('GC_QUOTE_API_CREDENTIALS')
# os.getenv('HTTP_PROXY')
# os.getenv('HTTPS_PROXY')

In [ ]:
# config = dotenv_values("../.env")
# key = config.get('GC_QUOTE_API_CREDENTIALS')
key = os.getenv('GC_QUOTE_API_CREDENTIALS')

In [ ]:

credentials = service_account.Credentials.from_service_account_file("../" + key)


In [ ]:

#os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
#os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
#os.environ['http_proxy'] = "http://127.0.0.1:20171"
#os.environ['https_proxy'] = "http://127.0.0.1:20171"



In [ ]:
os.environ

In [ ]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 20
		"""

In [ ]:

# query_job = client.query(QUERY)


In [ ]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

In [ ]:
df.head()

In [ ]:
QUERY = """
		select *
		from elife-data-warehouse-prod.ods.ride_ride as t0 
        limit 20
		"""


In [ ]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

In [ ]:
df.head()

In [ ]:
QUERY = """
		select
		    t0.id as ride_id,
		    t1.id as dispatch_id,
            CASE
                WHEN t2.trip_no = t1.trip_no THEN t2.trip_type
                ELSE NULL
            END AS trip_type_id,
		    ARRAY_TO_STRING(ARRAY(
		        SELECT
                    t3.name
                FROM
                    elife-data-warehouse-prod.ods.ride_enum 
                    AS t3
                    WHERE t3.id = t2.trip_type AND t2.trip_no = t1.trip_no
            ), ',') AS trip_type,
		    t1.trip_no,
		    t0.stat AS ride_status_id,
		    ARRAY_TO_STRING(ARRAY(
		        SELECT
                    t3.name
                FROM
                    elife-data-warehouse-prod.ods.ride_enum 
                    AS t3
                    WHERE t3.id = t0.stat
            ), ',') AS ride_status,
            CASE
                WHEN t3.id = t0.partner_tran_id THEN t3.partner_id
                ELSE NULL
            END AS partner_id,
		    t0.from_time_str,
		    t0.from_timezone_str,
		    t0.to_time_str,
		    t0.to_timezone_str,
		    t0.from_place_id,
		    ARRAY_TO_STRING(ARRAY(
		        SELECT
                    t1.name
                FROM
                    elife-data-warehouse-prod.ods.ride_place
                    AS t1
                    WHERE t1.id = t0.from_place_id
            ), ',') AS from_place,
		    t0.to_place_id,
		    ARRAY_TO_STRING(ARRAY(
		        SELECT
                    t1.name
                FROM
                    elife-data-warehouse-prod.ods.ride_place
                    AS t1
                    WHERE t1.id = t0.to_place_id
            ), ',') AS to_place,
		    t0.distance,
		    t0.duration,
		    ARRAY_TO_STRING(ARRAY(
		        SELECT
                    t1.name
                FROM
                    elife-data-warehouse-prod.ods.ride_vehicle_class
                    AS t1
                    WHERE t1.id = t0.vehicle_class_id
            ), ',') AS vehicle_class,
		    t0.vehicle_class_id,
		    t0.passenger_count,
		    t0.luggage_count,
		    t0.trip_count,
		    t0.children_count,
		    t0.infant_count,
		    t1.amount as dispatch_amount,
		    t1.currency as dispatch_currency,
		    
		FROM 
            elife-data-warehouse-prod.ods.ride_ride AS t0 
            JOIN elife-data-warehouse-prod.ods.ride_dispatch AS t1 ON t0.id = t1.ride_id 
            LEFT JOIN elife-data-warehouse-prod.ods.ride_trip AS t2 ON t0.id = t2.ride_id 
            LEFT JOIN elife-data-warehouse-prod.ods.ride_partner_tran AS t3 ON t0.partner_tran_id = t3.id
        WHERE
            from_time_str > '2024-01-01'
        LIMIT
            10
		"""


In [ ]:

#df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

In [ ]:
QUERY = """
        WITH
        TripTypeID AS (
            SELECT
                t1.ride_id,
                CASE
                    WHEN t2.trip_no = t1.trip_no THEN t2.trip_type
                    ELSE NULL
                END AS trip_type_id,
            FROM
                elife-data-warehouse-prod.ods.ride_dispatch AS t1
                JOIN elife-data-warehouse-prod.ods.ride_trip AS t2 ON t1.ride_id = t2.ride_id
        ),
        TripType AS (
            SELECT
                t1.ride_id,
                t1.trip_type_id,
                t2.name AS trip_type,
            FROM
                TripTypeID AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_enum AS t2 ON t1.trip_type_id = t2.id
        ),
        RideStatus AS (
            SELECT
                t1.id AS ride_id,
                t1.stat AS ride_status_id,
                t2.name AS ride_status,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_enum AS t2 ON t1.stat = t2.id
        ),
        PartnerID AS (
            SELECT
                t1.id AS ride_id,
                CASE
                    WHEN t2.id = t1.partner_tran_id THEN t2.partner_id
                    ELSE NULL
                END AS partner_id,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_partner_tran AS t2 ON t1.partner_tran_id = t2.id
        ),
        Partner AS (
            SELECT
                t1.ride_id,
                t1.partner_id,
                t2.name AS partner,
            FROM
                PartnerID AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_partner AS t2 ON t1.partner_id = t2.id
        ),
        FromPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.from_place_id as start_place_id,
                t2.name AS start_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.dim.dim_place AS t2 ON t1.from_place_id = t2.id
        ),
        ToPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.to_place_id as end_place_id,
                t2.name AS end_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.dim.dim_place AS t2 ON t1.to_place_id = t2.id
        ),
        VehicleClass AS (
            SELECT
                t1.id AS ride_id,
                t1.vehicle_class_id as vehicle_class_id,
                t2.name AS vehicle_class,
            FROM
                elife-data-warehouse-prod.ods.ride_ride AS t1
                LEFT JOIN elife-data-warehouse-prod.ods.ride_vehicle_class AS t2 ON t1.vehicle_class_id = t2.id
        ),
        PricingTraining AS (SELECT t0.id       AS ride_id,
                                   t1.id       AS dispatch_id,
                                   t2.trip_type_id,
                                   t2.trip_type,
                                   t1.trip_no,
                                   t3.ride_status_id,
                                   t3.ride_status,
                                   t4.partner_id,
                                   t4.partner,
                                   t0.from_time_str,
                                   t0.from_timezone_str,
                                   t0.to_time_str,
                                   t0.to_timezone_str,
                                   t5.start_place_id,
                                   t5.start_place,
                                   t5.longt    AS start_longt,
                                   t5.latit    AS start_latit,
                                   t6.end_place_id,
                                   t6.end_place,
                                   t6.longt    AS end_longt,
                                   t6.latit    AS end_latit,
                                   t0.distance,
                                   t0.duration,
                                   t7.vehicle_class_id,
                                   t7.vehicle_class,
                                   t0.passenger_count,
                                   t0.luggage_count,
                                   t0.trip_count,
                                   t0.children_count,
                                   t0.infant_count,
                                   t1.amount   as dispatch_amount,
                                   t1.currency as dispatch_currency,

                            FROM `elife-data-warehouse-prod.ods.ride_ride` AS t0
                                     JOIN `elife-data-warehouse-prod.ods.ride_dispatch` AS t1 ON t0.id = t1.ride_id
                                     LEFT JOIN TripType AS t2 ON t0.id = t2.ride_id
                                     LEFT JOIN RideStatus AS t3 ON t0.id = t3.ride_id
                                     LEFT JOIN Partner AS t4 ON t0.id = t4.ride_id
                                     LEFT JOIN FromPlace AS t5 ON t0.id = t5.ride_id
                                     LEFT JOIN ToPlace AS t6 ON t0.id = t6.ride_id
                                     LEFT JOIN VehicleClass AS t7 ON t0.id = t7.ride_id
                            WHERE from_time_str > '2024-01-01'
        )

        SELECT
            t.ride_id,
            t.dispatch_id,
            t.trip_type,
            t.trip_no,
            t.ride_status,
            t.partner,
            t.from_time_str,
            t.from_timezone_str,
            t.to_time_str,
            t.to_timezone_str,
            t.start_place,
            t.start_latit,
            t.start_longt,
            t.end_place,
            t.end_latit,
            t.end_longt,
            t.distance,
            t.duration,
            t.vehicle_class,
            t.passenger_count,
            t.luggage_count,
            t.trip_count,
            t.children_count,
            t.infant_count,
            t.dispatch_amount,
            t.dispatch_currency,

        FROM
            PricingTraining AS t
        WHERE
            from_time_str > '2024-01-01'
        LIMIT
            200


"""


In [ ]:
df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

In [ ]:
df

In [ ]:
from sqllineage.runner import LineageRunner
#from lineagex.lineagex import lineagex

In [ ]:
sql =   """
        WITH
        TripTypeID AS (
            SELECT
                t1.ride_id,
                CASE
                    WHEN t2.trip_no = t1.trip_no THEN t2.trip_type
                    ELSE NULL
                END AS trip_type_id,
            FROM
                `elife-data-warehouse-prod.ods.ride_dispatch` AS t1
                JOIN `elife-data-warehouse-prod.ods.ride_trip` AS t2 ON t1.ride_id = t2.ride_id
        ),
        TripType AS (
            SELECT
                t1.ride_id,
                t1.trip_type_id,
                t2.name AS trip_type,
            FROM
                TripTypeID AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_enum AS` t2 ON t1.trip_type_id = t2.id
        ),
        RideStatus AS (
            SELECT
                t1.id AS ride_id,
                t1.stat AS ride_status_id,
                t2.name AS ride_status,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_enum` AS t2 ON t1.stat = t2.id
        ),
        PartnerID AS (
            SELECT
                t1.id AS ride_id,
                CASE
                    WHEN t2.id = t1.partner_tran_id THEN t2.partner_id
                    ELSE NULL
                END AS partner_id,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_partner_tran` AS t2 ON t1.partner_tran_id = t2.id
        ),
        Partner AS (
            SELECT
                t1.ride_id,
                t1.partner_id,
                t2.name AS partner,
            FROM
                PartnerID AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_partner` AS t2 ON t1.partner_id = t2.id
        ),
        FromPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.from_place_id as start_place_id,
                t2.name AS start_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.dim.dim_place` AS t2 ON t1.from_place_id = t2.id
        ),
        ToPlace AS (
            SELECT
                t1.id AS ride_id,
                t1.to_place_id as end_place_id,
                t2.name AS end_place,
                t2.lng AS longt,
                t2.lat AS latit,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.dim.dim_place` AS t2 ON t1.to_place_id = t2.id
        ),
        VehicleClass AS (
            SELECT
                t1.id AS ride_id,
                t1.vehicle_class_id as vehicle_class_id,
                t2.name AS vehicle_class,
            FROM
                `elife-data-warehouse-prod.ods.ride_ride` AS t1
                LEFT JOIN `elife-data-warehouse-prod.ods.ride_vehicle_class` AS t2 ON t1.vehicle_class_id = t2.id
        ),
        PricingTraining AS (SELECT t0.id       AS ride_id,
                                   t1.id       AS dispatch_id,
                                   t2.trip_type_id,
                                   t2.trip_type,
                                   t1.trip_no,
                                   t3.ride_status_id,
                                   t3.ride_status,
                                   t4.partner_id,
                                   t4.partner,
                                   t0.from_time_str,
                                   t0.from_timezone_str,
                                   t0.to_time_str,
                                   t0.to_timezone_str,
                                   t5.start_place_id,
                                   t5.start_place,
                                   t5.longt    AS start_longt,
                                   t5.latit    AS start_latit,
                                   t6.end_place_id,
                                   t6.end_place,
                                   t6.longt    AS end_longt,
                                   t6.latit    AS end_latit,
                                   t0.distance,
                                   t0.duration,
                                   t7.vehicle_class_id,
                                   t7.vehicle_class,
                                   t0.passenger_count,
                                   t0.luggage_count,
                                   t0.trip_count,
                                   t0.children_count,
                                   t0.infant_count,
                                   t1.amount   as dispatch_amount,
                                   t1.currency as dispatch_currency,

                            FROM `elife-data-warehouse-prod.ods.ride_ride` AS t0
                                     JOIN `elife-data-warehouse-prod.ods.ride_dispatch` AS t1 ON t0.id = t1.ride_id
                                     LEFT JOIN TripType AS t2 ON t0.id = t2.ride_id
                                     LEFT JOIN RideStatus AS t3 ON t0.id = t3.ride_id
                                     LEFT JOIN Partner AS t4 ON t0.id = t4.ride_id
                                     LEFT JOIN FromPlace AS t5 ON t0.id = t5.ride_id
                                     LEFT JOIN ToPlace AS t6 ON t0.id = t6.ride_id
                                     LEFT JOIN VehicleClass AS t7 ON t0.id = t7.ride_id
                            WHERE from_time_str > '2024-01-01'
        )

        SELECT
            t.ride_id,
            t.dispatch_id,
            t.trip_type,
            t.trip_no,
            t.ride_status,
            t.partner,
            t.from_time_str,
            t.from_timezone_str,
            t.to_time_str,
            t.to_timezone_str,
            t.start_place,
            t.start_latit,
            t.start_longt,
            t.end_place,
            t.end_latit,
            t.end_longt,
            t.distance,
            t.duration,
            t.vehicle_class,
            t.passenger_count,
            t.luggage_count,
            t.trip_count,
            t.children_count,
            t.infant_count,
            t.dispatch_amount,
            t.dispatch_currency,

        FROM
            PricingTraining AS t
        WHERE
            from_time_str > '2024-01-01'
        LIMIT
            200
        """ 

In [ ]:
# result = LineageRunner("../data/pricing_training_bigquery.sql", dialect="bigquery")
result = LineageRunner(sql, dialect="bigquery")
#from lineagex.example import example
#example("dependency_example")
#lineagex(sql="../data/pricing_training_bigquery.sql", target_schema="schema1", search_path_schema="schema1, public")

In [ ]:
print(result)

In [ ]:
import numpy as np
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import plotly.express as px
from datetime import timedelta



In [ ]:
from geopy.distance import geodesic as GD
import haversine as hs
from haversine import Unit

In [ ]:
df.loc[:,['start_latit','start_longt','end_latit','end_longt']]
df.columns

In [ ]:
df['haversine_distance'] = df.apply(lambda x: hs.haversine((x['start_latit'],x['start_longt']),(x['end_latit'],x['end_longt']), unit=Unit.KILOMETERS), axis=1)
df['geodesic_distance'] = df.apply(lambda x: GD((x['start_latit'],x['start_longt']),(x['end_latit'],x['end_longt'])).km, axis=1)

In [ ]:
df.loc[:,['distance','haversine_distance','geodesic_distance','ride_status','trip_type','vehicle_class','partner']]


In [ ]:
df_trips = df.loc[:,['start_latit','start_longt','end_latit','end_longt', 'distance', 'duration', 'geodesic_distance', 'dispatch_amount', 'dispatch_currency', 'from_timezone_str']]
df_trips['saved_distance'] = df['distance']/1000.0
df_trips

In [ ]:
source = df_trips.loc[:,['start_latit','start_longt']]
destin = df_trips.loc[:,['end_latit','end_longt']]
source.loc[0]

In [ ]:
source.loc[0,'start_latit']

In [ ]:
start = f"{df_trips.loc[0,'start_longt']},{df_trips.loc[0,'start_latit']}"
end = f"{df_trips.loc[0,'end_longt']},{df_trips.loc[0,'end_latit']}"
start
end

In [ ]:
url = f'http://router.project-osrm.org/route/v1/driving/{start};{end}?alternatives=false&annotations=nodes'

headers = { 'Content-type': 'application/json'}
r = requests.get(url, headers = headers)
print("Calling API ...:", r.status_code) # Status Code 200 is success


In [ ]:

routejson = r.json()
route_nodes = routejson['routes'][0]['legs'][0]['annotation']['nodes']
route_distance = routejson["routes"][0]["distance"]
route_duration = routejson["routes"][0]["duration"]


In [ ]:
td = timedelta(seconds=route_duration)
td
route_duration

In [ ]:

f"{str(td)}"


In [ ]:
len(df)

In [ ]:
df_trips.loc[:, ['route_distance', 'route_duration','saved_duration']] = np.zeros(shape=(len(df),3))
df_trips['route_duration'] = df_trips['route_duration'].astype('object')
df_trips['saved_duration'] = df_trips['saved_duration'].astype('object') 
df_trips
df_trips.dtypes

In [ ]:

#td = timedelta(seconds=route_duration)
df.loc[0,'duration']
td = timedelta(seconds=float(df.loc[0,'duration']))
str(td)


In [ ]:
df_trips.loc[0,'start_longt']
df_trips.loc[0,'duration']
df_trips.head()

In [ ]:
from tqdm import tqdm
for i in tqdm(range(0, len(df_trips))):
    start = f"{float(df_trips.loc[i,'start_longt'])},{float(df_trips.loc[i,'start_latit'])}"
    end   = f"{float(df_trips.loc[i,'end_longt'])},{float(df_trips.loc[i,'end_latit'])}"
    url = f'http://router.project-osrm.org/route/v1/driving/{start};{end}?alternatives=false&annotations=nodes'
    headers = { 'Content-type': 'application/json'}
    try:
        r = requests.get(url, headers = headers)
    except requests.exceptions.RequestException as e:
        df_trips.loc[i,'route_distance'] = -100.0
        df_trips.loc[i,'route_duration'] = 'N/A' 
        df_trips.loc[i,'saved_duration'] = 'N/A'
        continue
        
    # print("Calling API ...:", r.status_code) # Status Code 200 is success
    routejson = r.json()
    route_dist = routejson["routes"][0]["distance"]
    route_dura = routejson["routes"][0]["duration"]
    df_trips.loc[i,'route_distance'] = route_dist/1000
    df_trips.loc[i,'route_duration'] = str(timedelta(seconds=route_dura))
    df_trips.loc[i,'saved_duration'] = str(timedelta(seconds=float(df_trips.loc[i,'duration'])))
    
#df['route_distance'] = df.apply(lambda x: get_route_distance((x['start_latit'],x['start_longt']),(x['end_latit'],x['end_longt'])), axis=1)  # not working since it is not vectorized  (need to create UFUNC)


In [ ]:
df_trips.loc[:,['distance','saved_duration','route_distance','route_duration']]

In [ ]:
df['route_duration'] = df_trips['route_duration']
df['route_distance'] = df_trips['route_distance']
df['saved_duration'] = df_trips['saved_duration']
df['saved_distance'] = df['distance']/1000.0



In [ ]:
df_trips['saved_distance'] = df['saved_distance']
df_trips

In [ ]:
### keeping every third element in the node list to optimise time
route_list = []
for i in range(0, len(route_nodes)):
    if i % 3==1:
        route_list.append(route_nodes[i])

coordinates = []

for node in tqdm(route_list):
    try:
        url = 'https://api.openstreetmap.org/api/0.6/node/' + str(node)
        r = requests.get(url, headers = headers)
        myroot = ET.fromstring(r.text)
        for child in myroot:
            lat, long = child.attrib['lat'], child.attrib['lon']
        coordinates.append((lat, long))
    except:
        continue
print(coordinates[:10])

In [ ]:
df_out = pd.DataFrame({'Node': np.arange(len(coordinates))})
df_out['coordinates'] = coordinates
df_out[['lat', 'long']] = pd.DataFrame(df_out['coordinates'].tolist())

In [ ]:

# Converting Latitude and Longitude into float
df_out['lat'] = df_out['lat'].astype(float)
df_out['long'] = df_out['long'].astype(float)

# Plotting the coordinates on map
color_scale = [(0, 'red'), (1,'green')]
fig = px.scatter_mapbox(df_out, 
                        lat="lat", 
                        lon="long", 
                        zoom=8, 
                        height=600,
                        width=900)


fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0})

In [ ]:
df.to_csv("../data/pricing_training_dataset_sample.csv")
df_trips.to_csv("../data/pricing_training_dataset_trip_sample.csv")